In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_ONE_Championship_events#Events"

soup = BeautifulSoup(
    requests.get(url).text,
    "html.parser"
)

In [3]:
table = soup.find_all("table", class_ = "sortable")[0]

In [4]:
table_rows = table.find_all("tr")[1:]

In [5]:
anchors = [_.find_all("a") for _ in table_rows if len(_.find_all("a"))>0]

In [6]:
import pandas as pd

cols = ["link", "name"]
df = pd.DataFrame([(anchor[0].get("href"), anchor[0].text) for anchor in anchors], columns=cols)

In [7]:
clean_link = lambda x: f"https://en.wikipedia.org/{x.split('#')[0]}"
df["link_clean"] = df["link"].apply(clean_link)

In [8]:
links = df["link_clean"].value_counts().reset_index().iloc[:,0]

In [9]:
links = links.to_list()

In [10]:
links = list(set([link for link in links if "one_" in link.lower()]))

In [15]:
def scrapeTable(url):
    soup = BeautifulSoup(requests.get(url).text)

    events = soup.find_all(
        lambda tag: tag.name == "span" and 
        ("one champ" in tag.text.lower() or "fight card" in tag.text.lower() or "results" in tag.text.lower())
        and tag.get("class") == ["mw-headline"]
        )
    
    dfs = []

    for event in events:
        df = pd.read_html(str(event.find_next("table", class_ = "toccolours")))[0]
        if df.empty:
            df = pd.read_html(str(event.find_next("table", class_ = "wikitable")))[0]
        df = df.assign(event_url = event.text)
        dfs.append(df)

    return pd.concat(dfs, ignore_index=True)

In [16]:
scrapeTable(links[0])

Main Card (Amazon Prime Video)                                          \
                        Weight class   Unnamed: 1_level_1 Unnamed: 2_level_1   
0             Bantamweight Muay Thai    Jonathan Haggerty               def.   
1                        Lightweight           Halil Amir               def.   
2                        Strawweight     Bokang Masunyane               def.   
3             Bantamweight Muay Thai          Felipe Lobo               def.   
4                 Women's Atomweight     Denice Zamboanga               def.   
5                       Welterweight         Isi Fitikefu               def.   
6                Women's Strawweight              Meng Bo               def.   
7  Catchweight (149.5 lbs) Muay Thai          Asa Ten Pow               def.   
8           Catchweight (149.25 lbs)  Jhanlo Mark Sangiao               def.   

                                                                             \
        Unnamed: 3_level_1                         Method Round  Time Notes   
0  Nong-O Gaiyanghadao (c)                   KO (punches)     1  2:40   [a]   
1            Maurice Abévi           Decision (unanimous)     3  5:00   NaN   
2            Hiroba Minowa           Decision (unanimous)     3  5:00   NaN   
3       Saemapetch Fairtex                   KO (punches)     3  1:56   NaN   
4          Julie Mezabarba           Decision (unanimous)     3  5:00   NaN   
5          Valmir da Silva  Submission (rear-naked choke)     1  4:15   NaN   
6           Dayane Cardoso           Decision (unanimous)     3  5:00   NaN   
7                Han Zihao                  TKO (punches)     3  0:52   NaN   
8         Matias Farinelli           Submission (kneebar)     1  0:58   NaN   

  event_url  
             
0   Results  
1   Results  
2   Results  
3   Results  
4   Results  
5   Results  
6   Results  
7   Results  
8   Results

In [17]:
dfs = []
for link in links:
    try:
        dfs.append(scrapeTable(link))
    except:
        print(link)

https://en.wikipedia.org//wiki/2023_in_ONE_Championship
https://en.wikipedia.org//wiki/ONE_Friday_Fights_8
https://en.wikipedia.org//wiki/2021_in_ONE_Championship
https://en.wikipedia.org//wiki/2020_in_ONE_Championship
https://en.wikipedia.org//wiki/ONE_Friday_Fights_15
https://en.wikipedia.org//wiki/ONE_Friday_Fights_14
https://en.wikipedia.org//wiki/ONE_Friday_Fights_11
https://en.wikipedia.org//wiki/2013_in_ONE_Championship
https://en.wikipedia.org//wiki/ONE_Friday_Fights_18
https://en.wikipedia.org//wiki/ONE_Friday_Fights_16
https://en.wikipedia.org//wiki/2014_in_ONE_Championship
https://en.wikipedia.org//wiki/2017_in_ONE_Championship
https://en.wikipedia.org//wiki/ONE_Friday_Fights_17
https://en.wikipedia.org//wiki/2018_in_ONE_Championship
https://en.wikipedia.org//wiki/ONE_Friday_Fights_13
https://en.wikipedia.org//wiki/2019_in_ONE_Championship
https://en.wikipedia.org//wiki/2022_in_ONE_Championship
https://en.wikipedia.org//wiki/2012_in_ONE_Championship
https://en.wikipedia.org/

In [19]:
soup = BeautifulSoup(requests.get("https://en.wikipedia.org//wiki/2023_in_ONE_Championship").text)

In [25]:
headers = soup.find_all(lambda tag: tag.name == "h2")

In [28]:
headers = [_ for _ in headers if _.find(lambda tag: tag.name == "span" and "one" in tag.text.lower()) != None]

In [49]:
headers = [
    {header: header.find_next(lambda tag: tag.name=="h3" and tag.find("span", id = "Results")!=None)}
    for header in headers]

In [60]:
headers = [header for header in headers if header[list(header)[0]] != None]

In [66]:
pd.read_html(str(list(headers[0].values())[0].findNext("table")))[0]

ONE Friday Fights 8 (YouTube / Facebook)  \
                               Weight Class   
0                       Flyweight Muay Thai   
1           Catchweight (128 lbs) Muay Thai   
2           Catchweight (116 lbs) Muay Thai   
3           Catchweight (113 lbs) Muay Thai   
4                    W.Atomweight Muay Thai   
5           Catchweight (116 lbs) Muay Thai   
6                    Bantamweight Muay Thai   
7                               Lightweight   
8           Catchweight (128 lb) Kickboxing   
9            Catchweight (118 lb) Muay Thai   
10                             Welterweight   
11                      Flyweight Muay Thai   

                                                            \
                     Unnamed: 1_level_1 Unnamed: 2_level_1   
0                Phetsukumvit Boybangna               def.   
1               Petlampun Muadablampang               def.   
2                NumSurin Chor.Ketweena               def.   
3             Banluerit Sitwatcharachai               def.   
4                        Junior Fairtex               def.   
5   Maisangkam Sor Yingcharoenkarnchang               def.   
6                         Mavlud Tupiev               def.   
7                   Vladimir Kanunnikov               def.   
8                          Huo Xiaolong               def.   
9                   Jomhod AutoMuayThai               def.   
10                   Salamat Orozakulov               def.   
11                     Mohammad Sadeghi               def.   

                                                                        
        Unnamed: 3_level_1                    Method Round  Time Notes  
0     Petchmuangsri Tded99              KO (punches)     2  2:36   NaN  
1      Rambong Sor.Terapat                KO (punch)     3  1:14   NaN  
2   Khunsuknoi Boomdeksian              KO (punches)     1  2:38   NaN  
3   Nuatoranee Jitmuangnon              KO (punches)     1  2:38   NaN  
4      Pancake Kiattongyot          Decision (split)     3  3:00   NaN  
5      Jomhot Charoenmuang            KO (left hook)     3  0:42   NaN  
6       Alaverdi Ramazanov      Decision (unanimous)     3  3:00   NaN  
7              Jason Ponet      Decision (unanimous)     3  5:00   NaN  
8        Seyed Mahdi Zadeh      Decision (unanimous)     3  3:00   NaN  
9        Daniel Gyllenberg  TKO (punches and elbows)     2  1:05   NaN  
10           Furqan Cheema      Decision (unanimous)     3  5:00   NaN  
11           Elias Ghazali      Decision (unanimous)     3  3:00   NaN